# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
***

# Import Modules <a name="import-modules"></a>

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

# Python Modules
import os
import sys

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_fe_graph"],
        "data",
        ),
    )

import pandas as pd
# pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np

# My Modules ******************************************************************
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot, Scaling_Relations_Plot

# Local Imports
from an_data_processing import load_df
from proj_data_fe_graph import proj_dir_name, system_color_map

# Script Inputs

In [2]:
save_plot = True
save_plot = False

bias = 0.


# Smart Formatting
smart_format = [

    # [
    #     {"spinpol": True},
    #     {"dash": "dot"},
    #     ],

    [
        {"system": "Fe_slab"},
        {"dash": "dashdot"},
        ],

    [
        {"system": "N_graph_Fe"},
        {"dash": "dot"},
        ],

    [
        {"system": "graph_Fe"},
        {"dash": "dash"},
        ],

    [
        {"system": "graphene"},
        {"dash": None},
        ],

    ]


In [3]:
%%capture
df_m = load_df(
    from_file=False,
    root_dir=os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow/adsorption_study"),
    )

# Exlcuding pdos and charge history columns from df

In [5]:
all_columns = set(list(df_m))

exclude_columns = set([
    "pdos_data",
    "charge_history",
    ])

columns_to_keep = list(all_columns - exclude_columns)

df_m = df_m[columns_to_keep]

# Instantiate the ORR_Free_E_Plot Instance

In [6]:
prop_name_list = [
    "system",
    "spinpol",
    "site",
    "graph_site",
     ]

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",  
    bias=bias,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="ORR",
    )

# Picking the Lowest Energy States

In [7]:
group_params = ["system", "spinpol", "graph_site"]
grouped = df_m.groupby(group_params)

lowest_energy_list = []
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    for j_cnt, df_j in df_i.groupby("adsorbate"):
        series_j = df_j.loc[df_j["ads_e"].idxmin()]
        lowest_energy_list.append(series_j)

df_2 = pd.concat(lowest_energy_list, axis=1, keys=[s.name for s in lowest_energy_list]).T

# Adding ORR data series

In [9]:
grouped = df_2.groupby(group_params)
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    prop_list_i = {}
    for prop_i in prop_name_list:    
        first_value = df_i[prop_i].tolist()[0]
        all_same_value = all([True if i == first_value else False for i in df_i[prop_i].tolist()])

        if all_same_value:
            prop_list_i[prop_i] = str(first_value)
        else:
            prop_list_i[prop_i] = str(None)

    name_i = "_".join(list(prop_list_i.values()))
    print(name_i)
    
    color_i = system_color_map[name_i]

    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
            opt_name=name_i,
            overpotential_type="ORR",
            color=color_i,
            )

Fe_slab_False_None_nan
Fe_slab_True_bridge_nan
N_graph_False_ring-center_nan
N_graph_True_None_nan
N_graph_Fe_False_None_C-trifold
N_graph_Fe_False_None_N-trifold
graph_Fe_False_None_nan
graph_Fe_True_None_nan
graphene_False_None_nan
graphene_True_None_nan


# Plotting <a name="fed_all"></a>

In [10]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

layout = ORR_PLT.plotly_fed_layout(
    plot_title="",
    plot_width=1.5 * 680,
    plot_height=1.5 * 510,
    )

py.plotly.iplot(
    {
        "data": ORR_PLT.plotly_data(),
        "layout": layout,
        },
        filename=os.path.join(save_dir, "pl_fed_all_systems"),
    )

In [11]:
# dat_lst_master = []

# for i_ind, group_i in enumerate(df.groupby("system")):
#     df_i = group_i[1]
    
#     # Tree Level Labels for group_i
#     tree_labels = df_i["tree_level_labels"].tolist()
#     tree_labels_2 = []
#     for item in tree_labels:
#         if sorted(item) not in tree_labels_2:
#             tree_labels_2.append(sorted(item))
#     assert len(tree_labels_2) == 1
#     tree_labels_2 = tree_labels_2[0]

#     # i_ind = i_ind * 2
#     color_list_i = color_list[i_ind:i_ind + 2]

# #     print(group_i[0])
    
#     dat_lst, layout = lowest_e_path(
#         df_i,
#         tree_labels_2,
#         color_list_i,
#         create_ideal_series=False,
#         opt_name=df_i["system"].iloc[0],
#         bias=bias,
#         manual_props="False",
# #         smart_format=smart_format,
#         smart_format=None,
#         )
#     dat_lst_master += dat_lst  

# # Adding Ideal ORR line
# ORR = ORR_Free_E_Plot()
# e_list_ideal = ORR.apply_bias(bias, ORR.ideal_energy)
# dat_ideal = ORR.create_plotly_series(
#     e_list_ideal,
#     group="Ideal",
#     name="Ideal",
#     color="red",
#     plot_mode="full_lines",
#     )

# dat_lst_master += dat_ideal

# dat_lst_master = reapply_colors(dat_lst_master)

# if save_plot:
#     save_dir = "01_fe_graph"
# else:
#     save_dir = "__temp__"

# py.iplot(
#     {
#         "data": dat_lst_master,
#         "layout": layout,
#         },
# #     filename="01_fe_graph/pl_fed_all_systems.html",
#     filename=os.path.join(save_dir, "pl_fed_all_systems"),
#     )

# from ase.visualize import view
# from ase.io.trajectory import Trajectory
# from ase import io
# # io.write("out.traj", df.loc[2].atoms_object)
# df_tmp = df[df["system"] == "N_graph_Fe"]

# images = df_tmp.iloc[0].atoms_object
# atoms = images[0]

# # io.write("out.traj", atoms)
# # view(images)

# atoms_i = df.iloc[0].atoms_object[-1]
# # view(atoms_i)


#     prop_list_i = {}
#     for prop_i in prop_name_list:    
#         first_value = df_i[prop_i].tolist()[0]
#         all_same_value = all([True if i == first_value else False for i in df_i[prop_i].tolist()])

#         if all_same_value:
#             prop_list_i[prop_i] = first_value
#         else:
#             prop_list_i[prop_i] = None

#     sys_i = prop_list_i["bulk_system"] + "_" + prop_list_i["facet"]
#     color_i = system_color_map_1.get(sys_i, "pink")    
            
#     if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
#         ORR_PLT.add_series(
#             df_i,
#             plot_mode="all",
#             opt_name=df_i["name_i"].tolist()[0],
#             overpotential_type="OER",
#             system_properties=prop_list_i,
# #             color=color_i,
#             )

# set(df["system"].tolist())

# df.loc[df["system"] == "N_graph_Fe", "spinpol"] = False

# prop_name_list = [
#     "system",
#     "spinpol",
#     "site",
#     "graph_site",
#     ]

# grouped = df_tmp.T.groupby(group_params)
# # lowest_energy_list = []
# for i_ind, (name, group) in enumerate(grouped):
#     print(name)
#     df_i = group
#     print(len(df_i))
#     prop_list_i = {}
#     for prop_i in prop_name_list:    
#         first_value = df_i[prop_i].tolist()[0]
#         all_same_value = all([True if i == first_value else False for i in df_i[prop_i].tolist()])

#         if all_same_value:
#             prop_list_i[prop_i] = str(first_value)
#         else:
#             prop_list_i[prop_i] = str(None)


# #     print(prop_list_i)
# #     sys_i = prop_list_i["bulk_system"] + "_" + prop_list_i["facet"]
# #     print(sys_i)
# #     color_i = system_color_map_1.get(sys_i, "pink")    

# "_".join(prop_list_i)

# prop_list_i

# "_".join(
#     list(prop_list_i.values())
#     )

# str(None)

# # df_2 = df_tmp.T
# # df_2[
# #     (df_2["system"] == "graph_Fe") & \
# #     (df_2["spinpol"] == True)
# #     ]